# **Penentuan Kelas Untuk Klasifikasi Faktor Kecelakaan Pesawat**

## Nama Kelompok

1.   Haarisah Yustika Putri Al-Jufri
2.   Ony Novianti
3.   Reynaldi Fakhri Pratama







In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pyspark
!pip install matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Import PySpark modules
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF
from pyspark.sql.functions import regexp_replace
from pyspark.ml.clustering import KMeans

# Inisialisasi SparkSession
spark = SparkSession.builder.appName("FlightAccidentStopWordsRemoval").getOrCreate()

# Load Data
data = spark.read.csv("/content/drive/MyDrive/Airplane_Crashes_and_Fatalities_Since_1908.csv", header=True, inferSchema=True)

# Hapus Baris Kosong
data = data.na.drop(subset=['Summary'])

# Hapus karakter tambahan seperti , . spasi
data = data.withColumn("Summary", regexp_replace(data["Summary"], "[\.,]", ""))

# Tokenize
tokenizer = Tokenizer(inputCol="Summary", outputCol="tokens")
data = tokenizer.transform(data)

# Hilangkan stop word
stopwords = StopWordsRemover.loadDefaultStopWords("english")
stopwords_remover = StopWordsRemover(inputCol="tokens", outputCol="filtered_tokens", stopWords=stopwords)
data = stopwords_remover.transform(data)

# Hasil data
# data.select("Summary", "filtered_tokens").show(truncate=False)

In [ ]:
# Tampil Kolom
# data.printSchema()


In [ ]:
# Jumlah baris data
data.count()


4878

In [ ]:
# Gabungkan hasil token-token menjadi satu
teks = [row[0] for row in data.select("filtered_tokens").collect()]

teks_gabung = [' '.join(dokumen) for dokumen in teks]

# print(teks_gabung)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# Membuat objek TfidfVectorizer
vectorizer = TfidfVectorizer()

# Menghitung nilai TF-IDF
tfidf_matrix = vectorizer.fit_transform(teks_gabung)

# Mendapatkan daftar fitur
feature_names = vectorizer.get_feature_names_out()
# Menampilkan hasil TF-IDF
# top_k = 3
# for i, row in enumerate(tfidf_matrix.toarray()):
#     print("Dokumen", i+1)
#     top_indices = np.argsort(row)[::-1][:top_k]
#     for index in top_indices:
#         print(feature_names[index])
#     print()


## Memanfaatkan Model LDA Untuk Penentuan Topik

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim import corpora, models, matutils

# Membuat corpus dari representasi TF-IDF
corpus = matutils.Sparse2Corpus(tfidf_matrix.T)

# Membangun kamus (dictionary)
dictionary = corpora.Dictionary.from_corpus(corpus, id2word=dict(enumerate(feature_names)))

# Melatih model LDA
num_topics = 9  # Jumlah topik yang diinginkan
lda_model = models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics)

# Menampilkan topik dan kata-kata kunci terkait
topics = lda_model.show_topics(num_topics=num_topics, num_words=5)
for topic in topics:
    print(topic)

# Menyematkan dokumen ke dalam topik
document = "Contoh dokumen yang ingin disematkan ke dalam topik"
bow_vector = dictionary.doc2bow(document.lower().split())
doc_topics = lda_model.get_document_topics(bow_vector)
for topic in doc_topics:
    print(topic)

(0, '0.013*"helicopter" + 0.008*"rebels" + 0.006*"crashed" + 0.006*"shot" + 0.006*"rotor"')
(1, '0.009*"crashed" + 0.009*"plane" + 0.007*"aircraft" + 0.007*"taking" + 0.007*"shortly"')
(2, '0.016*"crashed" + 0.011*"takeoff" + 0.009*"mountain" + 0.007*"poor" + 0.006*"weather"')
(3, '0.015*"approach" + 0.015*"plane" + 0.013*"crashed" + 0.012*"cargo" + 0.011*"runway"')
(4, '0.018*"route" + 0.018*"en" + 0.014*"crashed" + 0.010*"plane" + 0.006*"cargo"')
(5, '0.012*"crashed" + 0.011*"plane" + 0.008*"aircraft" + 0.006*"flight" + 0.005*"taking"')
(6, '0.009*"plane" + 0.009*"crashed" + 0.009*"landing" + 0.008*"pilot" + 0.008*"aircraft"')
(7, '0.009*"crashed" + 0.009*"flight" + 0.008*"pilot" + 0.008*"plane" + 0.008*"aircraft"')
(8, '0.010*"crashed" + 0.009*"mountain" + 0.009*"taking" + 0.008*"aircraft" + 0.008*"plane"')
(0, 0.11111111)
(1, 0.11111111)
(2, 0.11111111)
(3, 0.11111111)
(4, 0.11111111)
(5, 0.11111111)
(6, 0.11111111)
(7, 0.11111111)
(8, 0.11111111)


In [ ]:
coherence_model = CoherenceModel(model=lda_model, texts=teks, dictionary=dictionary, coherence='c_v')
coherence_score = coherence_model.get_coherence()

# Hasil coherence score
print("Coherence Score:", coherence_score)


Coherence Score: 0.3672366124986783
